In [10]:
import pandas as pd


In [11]:
!pwd

/Users/alexandre/ml_portfolio/ML_fin_project


In [12]:
data =pd.read_csv("data/data_bitcoin.csv")

In [19]:
data[data['content']== 'AA']

,language,url,date,content


In [ ]:
# Error c est quand l'article c'est un pdf (pas scrappable)

In [11]:
print(data["url"].iloc[2])

https://www.ecb.europa.eu/euro/digital_euro/timeline/profuse/shared/pdf/ecb.deprep241212_14erpb_Annex_to_preliminary_methodology_for_holding_limit_calibration.en.pdf


In [12]:
print(data["content"].iloc[2][:5000])
print("\nTaille de l'article (nombre de caractères):\n",len(data["content"].iloc[2]))

Error

Taille de l'article (nombre de caractères):
 5


In [6]:
import requests
from pdfminer.high_level import extract_text
from io import BytesIO

def extraire_texte_pdf_miner(lien):
        try:
            reponse = requests.get(lien)
            reponse.raise_for_status()

            fichier_pdf = BytesIO(reponse.content)
            texte = extract_text(fichier_pdf)
            return texte

        except requests.exceptions.RequestException as e:
            return f"Erreur lors du téléchargement du PDF : {e}"
        except Exception as e:
            return 'Error'

# Exemple d'utilisation
lien_pdf = "https://www.ecb.europa.eu/euro/digital_euro/timeline/profuse/shared/pdf/ecb.deprep241212_14erpb_Update_on_work_on_methodology_for_holding_limit_calibration.en.pdf"
texte = extraire_texte_pdf_miner(lien_pdf)
print(texte)


ECB-PUBLIC

Preliminary 
methodology for 
calibrating holding 
limits

ERPB meeting – Item 3

10 December 2024

Digital euro project

Purpose of today’s discussion

- The ECB is working to develop a methodology for setting digital euro holding 
limits, balancing user experience with monetary policy and financial stability 
implications. 

- This presentation provides a description of the tools being developed 
and aims to collect feedback from ERPB members on these tools.

Nota bene: this presentation and its annex have not been adopted or endorsed 
by ECB decision making bodies and therefore any views expressed shall not, 
under any circumstances, be regarded as an official position of the ECB. This 
content is intended solely to facilitate discussions and feedback by experts from 
the ERPB associations to be provided to the ECB digital euro project team.

2

www.ecb.europa.eu © 

Our framework of analysis

ECB-PUBLIC

Framework based on the objectives set out in Article 15.1 of the

In [32]:
import requests
import pandas as pd

def get_bce_key_rates():
    url = "https://sdw-wsrest.ecb.europa.eu/service/data/FM/B.U2.EUR.4F.KR.M"
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Erreur lors de la récupération des données.")
        return None
    data = response.json()
    structure = data['structure']['dimensions']['observation'][0]['values']
    series = data['dataSets'][0]['series']
    records = []
    for key, value in series.items():
        for date_index, observation in value['observations'].items():
            date = structure[int(date_index)]['id']
            records.append((date, observation[0]))
    df = pd.DataFrame(records, columns=["Date", "Rate"])
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", inplace=True)
    return df

rates_df = get_bce_key_rates()
if rates_df is not None:
    print(rates_df)
    rates_df.to_csv("bce_key_rates.csv")


Erreur lors de la récupération des données.


In [23]:
observations = rates_df['dataSets'][0]['series']['0:0:0:0:0']['observations']

In [33]:
data_rates = pd.read_csv('data/ECB Data Portal daily_20241213172734.csv')

In [34]:
data_rates

,DATE,TIME PERIOD,Deposit facility - date of changes (raw data) - Change in percentage points compared to previous rate (FM.B.U2.EUR.4F.KR.DFR.CHG),Deposit facility - date of changes (raw data) - Level (FM.B.U2.EUR.4F.KR.DFR.LEV),Marginal lending facility - date of changes (raw data) - Change in percentage points compared to previous rate (FM.B.U2.EUR.4F.KR.MLFR.CHG),Marginal lending facility - date of changes (raw data) - Level (FM.B.U2.EUR.4F.KR.MLFR.LEV),Main refinancing operations - irrespective of which type of rate fixed or variable (date of changes) - Change in percentage points compared to previous rate (FM.B.U2.EUR.4F.KR.MRR.CHG),Main refinancing operations - fixed rate tenders (fixed rate) (date of changes) - Level (FM.B.U2.EUR.4F.KR.MRR_FR.LEV),ECB Main refinancing operations - variable rate tenders (minimun bid rate) (date of changes) - Level (FM.B.U2.EUR.4F.KR.MRR_MBR.LEV),Deposit facility - date of changes (raw data) - Level (FM.D.U2.EUR.4F.KR.DFR.LEV),Main refinancing operations - fixed rate tenders (fixed rate) (date of changes) - Level (FM.D.U2.EUR.4F.KR.MRR_FR.LEV),Main refinancing operations - Minimum bid rate/fixed rate (date of changes) - Level (FM.D.U2.EUR.4F.KR.MRR_RT.LEV)
0,1999-01-01,01 Jan 1999,NaN,2.00,NaN,4.50,NaN,3.00,NaN,2.00,3.0,3.0
1,1999-01-02,02 Jan 1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,3.0,3.0
2,1999-01-03,03 Jan 1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,3.0,3.0
3,1999-01-04,04 Jan 1999,0.75,2.75,-1.25,3.25,NaN,3.00,NaN,2.75,3.0,3.0
4,1999-01-05,05 Jan 1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.75,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9475,2024-12-10,10 Dec 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.25,3.4,3.4
9476,2024-12-11,11 Dec 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.25,3.4,3.4
9477,2024-12-12,12 Dec 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.25,3.4,3.4
9478,2024-12-13,13 Dec 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.25,3.4,3.4


In [61]:
def get_related_topics(url):
    try:
        response = requests.get(url)
    except ValueError:
        return []

    if response.status_code != 200:
        print(f"Erreur lors de la récupération de la page : {response.status_code}")
        return []
    

        soup = BeautifulSoup(response.text, 'html.parser')


    related_topics = []
    for a_tag in soup.find_all('a', class_='taxonomy-tag'):
        topic = a_tag.get_text()
        related_topics.append(topic)
    
    return related_topics

from bs4 import BeautifulSoup
import requests

In [62]:
data.iloc[0:1]['url'].apply(lambda x : get_related_topics(x))

0    []
Name: url, dtype: object

In [63]:
data['url']= data['url'].apply(lambda x : get_related_topics(x))

ConnectionError: HTTPSConnectionPool(host='www.ecb.europa.eu20240701_ecb_forum_on_central_banking.en.html', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x12a40c440>: Failed to resolve 'www.ecb.europa.eu20240701_ecb_forum_on_central_banking.en.html' ([Errno 8] nodename nor servname provided, or not known)"))